In [1]:
from vllm import LLM, SamplingParams
import json

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-28 08:31:14,387	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# 初始化 vLLM 的离线推理引擎，这里选择的是 "/root/chinese-llama2" 模型
# model_path = 'Qwen-main/output_qwen/Final/Qwen1.5_y_3e-5'
model_path = 'Qwen-main/input_qwen/qwen1.5-7B'
llm = LLM(model=model_path)

INFO 06-28 08:31:15 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='Qwen-main/input_qwen/qwen1.5-7B', tokenizer='Qwen-main/input_qwen/qwen1.5-7B', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-28 08:31:15 selector.py:45] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 06-28 08:31:15 selector.py:21] Using XFormers backend.
INFO 06-28 08:31:20 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 06-28 08:31:22 gpu_executor.py:94] # GPU blocks: 6844, # CPU blocks: 512
INFO 06-28 08:31:24 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-28 08:31:24 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-28 08:31:27 model_runner.py:867] Graph capturing finished in 4 secs.


In [3]:
# 读取数据
data_path = "Qwen-main/data/eval_new.json"

# 读取json文件内部的内容
f = open(data_path)
content = f.read()
before_prompts = json.loads(content)

prompts = []
y = []
for i in range(len(before_prompts)):
    prompt = before_prompts[i]['conversations'][0]['value'] + ':'
    prompts.append(prompt)
    y.append(before_prompts[i]['conversations'][1]['value'])

print(len(y))

785


In [4]:
pmts = []
for i in range(len(prompts)):
    pmt = "摘要案例为为：{}, 请按照案例格式回答下面问题：{}".format(y[0], prompts[i])
    pmts.append(pmt)

In [5]:
# 定义采样参数，temperature 控制生成文本的多样性，top_p 控制核心采样的概率
sampling_params = SamplingParams(top_p=0.5, max_tokens=900)
output = llm.generate(pmts, sampling_params)

Processed prompts: 100%|██████████| 785/785 [09:49<00:00,  1.33it/s]


In [16]:
output[0].outputs[0].text

' 原告广州摩安珂服饰有限公司与被告现想商贸（上海）有限公司、现想商贸（上海）有限公司深圳第二分公司租赁合同纠纷一案。原告提出诉求：判令两被告向原告支付拖欠的款项及利息；两被告对上述债务承担连带清偿责任。被告未到庭，未提交书面意见及证据。经查明原告与被告现想公司深圳第二分公司签署的《专柜联营（租赁）合作协议》和《补充协议》均为双方的真实意思表示，内容未违反法律及行政法规的强制性规定，为有效合同。被告现想公司深圳第二分公司是现想公司设立的分支机构，在法律上、经济上没有独立性，仅仅是总公司的附属机构，不具有企业法人资格，其民事责任由总公司承担。依照《中华人民共和国合同法》第四十四条第一款、第六十条第一款、第一百零七条、第一百零九条、第一百一十二条，《中华人民共和国公司法》第十四条第一款以及《中华人民共和国民事诉讼法》第六十四条第一款、第一百四十二条的规定，判决: 一、被告现想商贸（上海）有限公司深圳第二分公司应向原告广州摩安珂服饰有限公司支付欠款及利息。 二、被告现想商贸（上海）有限公司对被告现想商贸（上海）有限公司深圳第二分公司不能清偿部分承担补充清偿责任。 三、驳回原告广州摩安珂服饰有限公司其他诉讼请求。'

In [6]:
## 将推理到的数据和原数据整合
predict_y = []
for i in range(len(output)):
    dict_={}
    dict_['y'] = y[i]
    if(len(output[i].outputs[0].text)!=0):
        dict_['pred'] = output[i].outputs[0].text
    else:
        dict_['pred'] = "本次生成失败"
    predict_y.append(dict_)
    
print("整合完成")

整合完成


In [8]:
json_file_path = "Qwen-main/data/eval_data/JI_FEW_0628_V2340.json"
json_file = open(json_file_path, mode='w', encoding='utf-8')
json.dump(predict_y, json_file, ensure_ascii=False)